In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from finance_byu.regtables import Regtable
import warnings


In [3]:
#Reads the Parquet into a df
df = pd.read_parquet('Data/BoQ_Data.parquet')

In [4]:
#This gives me a list of all the different dates in ascending order
difDates = df["DATE"].tolist()
difDates= list(set(difDates))
difDates.sort()

In [5]:
print(difDates[653])

2009-06-30 00:00:00


In [6]:
#Sets the Vars you want to test
vars_list = ["prof","roic","BEME","ME","p1_mktrf_beta"]

In [7]:

#Generates a set of lists that have values you can test. For EX, some of the early dates don't have all the vars youll want and because we drop nan they wont work
works = []
noworks = []
for date in difDates:
    Datedf = df[(df["DATE"] == date)]
    dataframe = Datedf[vars_list + ['RET']]
    dataframe = dataframe.dropna().reset_index(drop=True)
    if len(dataframe) == 0:
        noworks.append(date)
    else:
        works.append(date)

#Helps you see the frist date you can start testing
print(works[0:10])

#This is the start and end date that works
startnend = (works[0],works[-1])

#Change them to a tuple so we can quickly and effeciently check if a date is in
works = tuple(works)
noworks = tuple(noworks)

[Timestamp('1956-07-31 00:00:00'), Timestamp('1956-08-31 00:00:00'), Timestamp('1956-09-30 00:00:00'), Timestamp('1956-10-31 00:00:00'), Timestamp('1956-11-30 00:00:00'), Timestamp('1956-12-31 00:00:00'), Timestamp('1957-01-31 00:00:00'), Timestamp('1957-02-28 00:00:00'), Timestamp('1957-03-31 00:00:00'), Timestamp('1957-04-30 00:00:00')]


In [17]:
#Creates a new row representing the RET for the next month represented by RET_MONTH
df['RET_MONTH'] = df['RET'].shift(-1)      

#This iterates through every row and removes the last RET_MONTH for every company, otherwise it will take the data of the first month from the next company
for i in range(df.shape[0]-1):
     if df.iloc[i, df.columns.get_loc("PERMNO")] != df.iloc[i + 1, df.columns.get_loc("PERMNO")]:
        df.at[i, "RET_MONTH"] = np.nan

df.at[df.shape[0], "RET_MONTH"] = np.nan

In [18]:
#Saves this to a df of your choosing
df.to_parquet('updated_BOQ_data.parquet')

In [8]:
dfupdate = pd.read_parquet('updated_BOQ_data.parquet')

In [3]:
import import_ipynb
import format_data

ModuleNotFoundError: No module named 'format_data'

In [20]:
#This gives me the regression of just 1 month
def Month_Regression(vars_list,month):
    #Check to make sure that regresion can be performed
    if month in noworks:
        print("NOT VALID MONTH")
        return 0
    
    #This grabs the data from the requested month
    Datedf = dfupdate[(dfupdate["DATE"] == month)]
    dataframe = Datedf[vars_list + ['RET_MONTH']]
    dataframe = dataframe.dropna().reset_index(drop=True)
    
    #We set our y to be the return of next month
    y = dataframe['RET_MONTH']
    del dataframe['RET_MONTH']
    reg_list = []

    #we set our x to be our desires variables
    X = dataframe
    X = sm.add_constant(X)

    #we make the regression 
    model = sm.OLS(y, X).fit()
    reg_list.append(model)
    #tbl = Regtable(reg_list,stat='tstat',sig='coeff')
    return reg_list

In [21]:
#This takes in a regression and titlle and spits out an exel file
def display(reg, title, filename):
    tbl = Regtable(reg, stat='tstat', sig='coeff')
    df_coeffs = tbl.render()
    df_coeffs.columns = [title]
    df_coeffs.to_excel(filename)

In [22]:

#This performs the regression from month to month
regression = Month_Regression(vars_list,difDates[199])
title = ['199']
for i in range(200,len(difDates)):
    reg = Month_Regression(vars_list,difDates[i])
    regression += reg
    title.append(i)

#How many months
print(len(title))

629


In [23]:
def reg_of_months(start, end, vars_list):
    #This finds the start and end index of the dates in the difDates var
    stindex = -1
    enindex = -1
    
    i = 0
    while stindex == -1:
        if difDates[i] == start:
            stindex = i
        else:
            i += 1

    while enindex == -1:
        if difDates[i] == end:
            enindex = i
        else:
            i += 1

    #performs the regression    
    regression = Month_Regression(vars_list,difDates[stindex])
    title = [stindex]
    for i in range(stindex+1,enindex+1):
        reg = Month_Regression(vars_list,difDates[i])
        regression += reg
        title.append(i)

    return regression, title
    

In [24]:
print(startnend)

(Timestamp('1956-07-31 00:00:00'), Timestamp('2023-12-31 00:00:00'))


In [25]:
#This gives me the reg of all months over time
reg, til = reg_of_months(startnend[0], startnend[1],vars_list)

In [26]:
#This transfers it to a Excel
display(reg, til,'all_time.xlsx')

In [27]:
#This is a data frame showing all of the coefficents over time

tbl = Regtable(reg, stat='tstat', sig='coeff')
coeff_view = tbl.get_coefficients()
coeff_view = coeff_view.T


In [36]:
#Does the same thing but just from 1/21 on
reg1, til1 = reg_of_months(difDates[653], startnend[1],vars_list)

tbl1 = Regtable(reg1, stat='tstat', sig='coeff')
coeff_view1 = tbl1.get_coefficients()
coeff_view1 = coeff_view1.T